In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q qdrant-client sentence-transformers

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
QDRANT_API_KEY = user_secrets.get_secret("qdrant")
QDRANT_HOST = "07961b0b-8262-43ec-9fcb-1552dd055ebd.us-east4-0.gcp.cloud.qdrant.io"  # Replace with your actual host

# Connect to Qdrant Cloud
client = QdrantClient(
    url=f"https://{QDRANT_HOST}",
    api_key=QDRANT_API_KEY,
)

In [ ]:
df = pd.read_csv("/kaggle/input/bangkok-traffy/bangkok_traffy.csv")
selected_column = ['type','organization','comment','coords']
df = df[selected_column]
df.info()

In [ ]:
print(df.isna().sum())
df = df.dropna().reset_index(drop=True)
print(df.isna().sum())

In [ ]:
def parse_coords(coord_str):
    try:
        lon, lat = map(float, coord_str.split(','))
        if -180 <= lon <= 180 and -90 <= lat <= 90:
            return [lon, lat]
        else:
            return np.nan
    except:
        return np.nan
def convert_type_string(s):
    if not isinstance(s, str):
        return np.nan
    try:
        items = s.strip("{}").split(",")
        cleaned = [item.strip() for item in items if item.strip()]
        return cleaned if cleaned else np.nan
    except Exception:
        return np.nan

In [ ]:
new_df = df.copy()


new_df['coords'] = new_df['coords'].apply(parse_coords)
new_df = new_df.dropna(subset=['coords']).reset_index(drop=True)


org_counts = new_df['organization'].value_counts()
valid_orgs = org_counts[org_counts >= 1000].index
df_filtered = new_df[new_df['organization'].isin(valid_orgs)]
df_filtered = df_filtered.reset_index(drop=True)


df_filtered['type'] = df_filtered['type'].apply(convert_type_string)


df_filtered = df_filtered.dropna(subset=['type']).reset_index(drop=True)


final_df = df_filtered.copy()


final_df.info()

In [ ]:
final_df.head()

In [ ]:
# Flatten the 'type' lists and get unique values
unique_values = set([item for sublist in final_df['type'] for item in sublist])

# Convert to list if desired
unique_values_list = list(unique_values)

# Print the unique values
print(unique_values_list)

In [ ]:
import matplotlib.pyplot as plt

# Step 1: Create a new column with the length of the 'type' list
final_df['type_len'] = final_df['type'].apply(len)

# Step 2: Count how many rows have each type length
type_len_counts = final_df['type_len'].value_counts().sort_index()

# Step 3: Plot the counts as a bar chart
plt.figure(figsize=(8, 5))
type_len_counts.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Number of Rows by Length of "type" List')
plt.xlabel('Length of type list')
plt.ylabel('Number of rows')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

# One-hot encode the 'type' list column directly
mlb = MultiLabelBinarizer()
type_onehot = mlb.fit_transform(final_df['type'])

# Create a DataFrame from the one-hot matrix
type_onehot_df = pd.DataFrame(type_onehot, columns=mlb.classes_)

# Drop the original 'type' column and concatenate the one-hot columns
final_df_onehot = pd.concat([final_df.drop(columns=['type','type_len']).reset_index(drop=True),
                             type_onehot_df], axis=1)

# Optional: View result
final_df_onehot.head()

In [ ]:
final_df_onehot.to_csv('/kaggle/working/final_df.csv')